In [6]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [7]:
spotify= pd.read_csv('spotify_limpio.csv')

In [8]:
spotify.dtypes

Artist             object
Track              object
Album              object
Album_type         object
Stream            float64
Danceability      float64
Energy            float64
Key                 int64
Key_name           object
Loudness          float64
Speechiness       float64
Acousticness      float64
Liveness          float64
Valence           float64
Tempo             float64
Tempo_category     object
Duration_ms       float64
Licensed             bool
official_video       bool
dtype: object

In [9]:
spotify.head()

,Artist,Track,Album,Album_type,Stream,Danceability,Energy,Key,Key_name,Loudness,Speechiness,Acousticness,Liveness,Valence,Tempo,Tempo_category,Duration_ms,Licensed,official_video
0,Cults,Gilded Lily,Offering,album,83194819.0,0.400,0.571,11,B,-6.534,0.0288,0.4680,0.5190,0.146,61.859,Adagio,212736.0,True,True
1,Los Pericos,Runaway,Pericos & Friends,album,101408935.0,0.676,0.731,9,A,-5.253,0.0368,0.0438,0.6240,0.812,150.151,Allegro,233268.0,False,False
2,Richard Marx,Satisfied,Repeat Offender,album,4281404.0,0.572,0.914,8,G# / Ab,-8.436,0.0311,0.0717,0.3190,0.843,108.991,Andante,254467.0,False,True
3,The Supremes,Come See About Me,Where Did Our Love Go,album,70357721.0,0.732,0.506,7,G,-11.735,0.0462,0.6880,0.1460,0.867,126.325,Allegro,163093.0,False,False
4,El Fantasma,Equipo RR,Equipo RR,single,13232046.0,0.755,0.550,7,G,-6.102,0.1070,0.6180,0.0868,0.926,114.501,Moderato / Allegretto,159976.0,False,False


In [10]:
spotify_sin_string= spotify.drop(['Artist', 'Track', 'Album', 'Album_type', 'Licensed', 'official_video', 'Tempo_category', 'Key_name'], axis=1)

## Modelo Baseline

Entrenamos regresión lineal utilizando las variables
disponibles sin aplicar transformaciones ni escalado. Los coeficientes obtenidos
presentan valores elevados, lo cual es esperable debido a la gran escala de la variable
objetivo (Stream) y de algunas variables explicativas.

In [11]:
X = spotify_sin_string.drop('Stream', axis=1)
y = spotify_sin_string['Stream']

model = LinearRegression()
model.fit(X, y)

y_pred_sklearn = model.predict(X)

In [12]:
print("\nParámetros (sklearn):")
print(f"w1 = {model.coef_[0]:.3f}, w2 = {model.coef_[1]:.3f}, b = {model.intercept_:.3f}")


Parámetros (sklearn):
w1 = 66604643.504, w2 = -155001782.610, b = 344807546.073


In [13]:
# Mostrar MSE final de ambos modelos
print("MSE (sklearn):", mean_squared_error(y, y_pred_sklearn))

MSE (sklearn): 5.963421418043348e+16


El error cuadrático medio (MSE) obtenido es muy alto, lo que se explica por el hecho de
que el error se eleva al cuadrado y la variable objetivo (stream) se mide en millones. Este resultado no se interpreta como un buen rendimiento, sino como una
referencia inicial que servirá para comparar modelos posteriores tras aplicar validación cruzada.

### Validación cruzada

El modelo baseline se evalúa mediante validación cruzada de 5 folds sobre el conjunto de
entrenamiento. Dado que el problema es de regresión, se utiliza el RMSE como métrica de
evaluación

In [14]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(
    model,
    X,
    y,
    cv=5,
    scoring="neg_root_mean_squared_error"
)

rmse_scores = -scores

print("RMSE por fold:", rmse_scores)
print("RMSE medio:", rmse_scores.mean())

RMSE por fold: [2.39088253e+08 2.40898993e+08 2.24886446e+08 2.44921291e+08
 2.70013995e+08]
RMSE medio: 243961795.48774132


Se obtiene un RMSE medio de
aproximadamente 244 millones de reproducciones. Esto implica que, de media, las
predicciones del modelo se desvían en ese orden de magnitud respecto al valor real.
Dado que la variable objetivo presenta valores muy elevados y una distribución muy
asimétrica, este resultado es esperable.

Usamos este valor como referencia para compararlo y ver si coseguimos reducirlo.

### Feature engineering

Probamos usando log(Stream) porque la popularidad musical tiene una distribución de cola larga,
y el log reduce la influencia de valores extremos y mejora el comportamiento del modelo.

No hay desequilibrio de clases, pero sí una distribución del target muy asimétrica,
típica de problemas de popularidad por lo que se aplican transformaciones como el logaritmo

Dado que el dataset no presenta una dimensión temporal,
no se aplican técnicas de agregación temporal ni lags.

In [15]:
import numpy as np

spotify_sin_string["log_stream"] = np.log1p(spotify_sin_string["Stream"])


In [16]:
X = spotify_sin_string.drop(["Stream", "log_stream"], axis=1)
y = spotify_sin_string["log_stream"]


Función para evaluar un modelo con validación cruzada (RMSE)

In [17]:
def baseline_model(X, y, cv=5):

    model = LinearRegression()
    
    scores = cross_val_score(
        model,
        X,
        y,
        cv=cv,
        scoring="neg_root_mean_squared_error"
    )
    
    rmse_scores = -scores
    
    print("RMSE por fold:", rmse_scores)
    print("RMSE medio:", rmse_scores.mean())
    
    return rmse_scores.mean()


Normalizamos las variables
numéricas restantes que presentan escalas muy distintas, como Loudness, Tempo y
Duration_ms. Esta estandarización permite mejorar la estabilidad del modelo lineal y
evitar que variables con mayor rango dominen el proceso de entrenamiento.

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

cols_to_scale = ["Loudness", "Tempo", "Duration_ms"]

spotify_sin_string[cols_to_scale] = scaler.fit_transform(
    spotify_sin_string[cols_to_scale]
)

In [19]:
baseline_model(X,y,5)

RMSE por fold: [1.5843802  1.62410519 1.58601829 1.60231204 1.62798889]
RMSE medio: 1.6049609213875864


np.float64(1.6049609213875864)

La normalización de las variables
explicativas no produce mejoras significativas en el error. Vamos a estudiar si las variables son muy relevantes

Analizamos la relevancia individual de
algunas variables.

In [20]:
X_sin_loudness = spotify_sin_string.drop(["Stream", "log_stream", 'Loudness'], axis=1)
y = spotify_sin_string["log_stream"]

In [21]:
baseline_model(X_sin_loudness,y,5)


RMSE por fold: [1.6079969  1.64845561 1.60442988 1.61219702 1.64655057]
RMSE medio: 1.6239259959896923


np.float64(1.6239259959896923)

Al eliminar *Loudness*, el RMSE
medio aumenta respecto al modelo base, lo que indica que esta variable aporta
información relevante al modelo. Por tanto, se decide mantener *Loudness* dentro del
conjunto final de variables.

In [22]:
X_sin_tempo = spotify_sin_string.drop(["Stream", "log_stream", 'Tempo'], axis=1)
y = spotify_sin_string["log_stream"]

In [23]:
baseline_model(X_sin_tempo,y,5)

RMSE por fold: [1.58397773 1.62445476 1.58651426 1.60334811 1.6285658 ]
RMSE medio: 1.60537213290043


np.float64(1.60537213290043)

Probamos eliminar *Tempo* y vemos que no aporta
ninguna mejora en el rendimiento, la mantenemos

ahora evaluamos el
impacto de eliminar simultáneamente las variables *Liveness* y *Duration_ms*

In [24]:
X_sin_liv_inst = spotify_sin_string.drop(["Stream", "log_stream", 'Liveness', 'Duration_ms'], axis=1)
y = spotify_sin_string["log_stream"]

In [25]:
baseline_model(X_sin_liv_inst,y,5)

RMSE por fold: [1.58749253 1.6274402  1.58581421 1.60222778 1.62841064]
RMSE medio: 1.606277074269362


np.float64(1.606277074269362)

El RMSE medio no mejora respecto al modelo base

In [26]:
X_sin_danc = spotify_sin_string.drop(["Stream", "log_stream", 'Danceability'], axis=1)
y = spotify_sin_string["log_stream"]

In [27]:
baseline_model(X_sin_danc,y,5)

RMSE por fold: [1.58385547 1.62483422 1.58681772 1.60327974 1.62742916]
RMSE medio: 1.6052432617400298


np.float64(1.6052432617400298)

Tampoco mejora al eliminar Danceability

se evalúa la eliminación
simultánea de *Speechiness* y *Acousticness*

In [28]:
X_sin_spee_aco = spotify_sin_string.drop(["Stream", "log_stream", 'Speechiness', 'Acousticness'], axis=1)
y = spotify_sin_string["log_stream"]

In [29]:
baseline_model(X_sin_spee_aco,y,5)

RMSE por fold: [1.58914758 1.64851021 1.59862188 1.62247755 1.65036117]
RMSE medio: 1.62182367807171


np.float64(1.62182367807171)

El RMSE medio aumenta de forma más notable respecto al modelo base, lo que
indica que la exclusión de estas variables perjudica el rendimiento del modelo. Por
tanto, se consideran relevantes y se mantienen.

probamos eliminando la variable key

In [31]:
X_sin_key = spotify_sin_string.drop(["Stream", "log_stream","Key"], axis=1)
y = spotify_sin_string["log_stream"]

In [32]:
baseline_model(X_sin_key,y,5)

RMSE por fold: [1.58436844 1.62421161 1.58621619 1.60239725 1.62820163]
RMSE medio: 1.6050790236833437


np.float64(1.6050790236833437)

A lo largo del proceso iterativo y selección de variables, se han evaluado
distintas combinaciones eliminando tanto variables individuales como pares de variables.
En todos los casos analizados, la eliminación de variables conduce a un empeoramiento o
no mejora del RMSE obtenido mediante validación cruzada.

En consecuencia, **se concluye que todas las variables consideradas aportan información
relevante al modelo** y se decide mantener el conjunto completo de predictores para las
siguientes etapas de modelado. Este resultado sugiere que la mejora del rendimiento
deberá venir de modelos más complejos en lugar de una reducción adicional del conjunto
de variables.